# DEAP

In [1]:
###libraries

#Data manipulation
import pandas as pd
import numpy as np
import pickle
import json
from sklearn import preprocessing

#Time series transformers
from pyts.multivariate.classification import MultivariateClassifier
from pyts.multivariate.transformation import WEASELMUSE
from pyts.classification import KNeighborsClassifier
from pywt import wavedec
import pyeeg

#Classifiers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

#Deep Learning
import mcfly

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

#Auxiliary Functions
from aux_functions import ResampleLinear1D
from aux_functions import get_features

#System Settings
import sys
import os

np.random.seed(42)

In [10]:
###Read the data
dataset = 'DEAP'
response = 'y_valence'

#Train and testing datasets
with open(os.path.join('Datasets_clean', 'DEAP', 'clean_data.pkl'), 'rb') as handle:
    clean_data = pickle.load(handle)

#Create train/test splits for cross-validation
subjects = np.array(list(clean_data.keys()))

train_index = []
test_index = []

skf = KFold(n_splits=10)
for train, test in skf.split(subjects):
    train_index.append(train)
    test_index.append(test)
    
#Optimal Hyperparameters
with open('hyperparameters.json') as json_file:
    hyperparameters = json.load(json_file)[dataset]
    
#Create scores dict
scores = {}

### Deep Learning

In [6]:
#Function to train the Neural Networks
def dl_func(dl_type,
            X_train_dl, 
            y_train_dl,
            X_val_dl,
            y_val_dl,
            X_test_dl,
            y_test_dl):
    
    #Validate diferent architectures
    num_of_candidate_models = 10
    random_search_epoches = 100
    random_search_es = 30
    best_model_epoches = 200
    best_model_es = 30
    
    for mod_type in [dl_type]:

        #Create architectures
        num_classes = y_train_dl.shape[1]
        metric = 'accuracy'
        models = mcfly.modelgen.generate_models(X_train_dl.shape,
                                                number_of_classes=num_classes,
                                                number_of_models = num_of_candidate_models,
                                                model_types = [mod_type])

        #Save intermediate results
        resultpath = 'temp'
        outputfile = os.path.join(resultpath, 'modelcomparison_{}_{}.json'.format(mod_type,dataset))

        #Find best architecture
        histories, val_accuracies, val_losses = mcfly.find_architecture.train_models_on_samples(X_train_dl, y_train_dl,
                                                                                                X_val_dl, y_val_dl,
                                                                                                models,
                                                                                                nr_epochs=random_search_epoches,
                                                                                                subset_size=None,
                                                                                                verbose=False,
                                                                                                outputfile=outputfile,
                                                                                                early_stopping_patience=random_search_es
                                                                                                )

        #Select and train best architecture
        best_model_index = np.argmax(val_accuracies)
        best_model, best_params, best_model_types = models[best_model_index]

        es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=best_model_es)
        mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

        history = best_model.fit(X_train_dl, y_train_dl,
                                 epochs=best_model_epoches, validation_data=(X_val_dl, y_val_dl),
                                 callbacks = [es,mc])

        #Accuracy in test dataset
        saved_model = load_model('best_model.h5')
        return saved_model.evaluate(X_test_dl, y_test_dl, verbose = False)[1]

In [16]:
#Train CNN and InceptionTime architectures
accuracies_cnn = []
accuracies_it = []

for train_ind, test_ind in zip(train_index, test_index):
    X_train = np.vstack([clean_data[key]['X'] for key in subjects[train_ind]])
    y_train = np.hstack([clean_data[key][response] for key in subjects[train_ind]])
    
    X_test = np.vstack([clean_data[key]['X'] for key in subjects[test_ind]])
    y_test = np.hstack([clean_data[key][response] for key in subjects[test_ind]])

    #Create train, validation and test sets for DL
    LE = LabelEncoder()
    skf_train_val = StratifiedKFold(n_splits=5)

    for train_index, val_index in skf_train_val.split(X_train, y_train):
        train_index = train_index
        val_index = val_index

    X_train_val = X_train.copy()

    X_train_dl = X_train_val[train_index].copy()
    X_train_dl = np.transpose(X_train_dl, (0,2,1))
    y_train_dl = y_train[train_index].copy()
    y_train_dl = to_categorical(LE.fit_transform(y_train_dl))

    X_val_dl = X_train_val[val_index].copy()
    X_val_dl = np.transpose(X_val_dl, (0,2,1))
    y_val_dl = y_train[val_index].copy()
    y_val_dl = to_categorical(LE.transform(y_val_dl))

    X_test_dl = np.transpose(X_test, (0,2,1))
    y_test_dl = to_categorical(LE.transform(y_test))
    
    #Define the networks
    acc_cnn = dl_func('CNN',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)
    
    acc_it = dl_func('InceptionTime',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)

    accuracies_cnn.append(acc_cnn)
    accuracies_it.append(acc_it)

#Save the mean and std of accuracies of the models over the splits
scores['dl_cnn'] = [np.mean(accuracies_cnn), np.std(accuracies_cnn)]
scores['dl_it'] = [np.mean(accuracies_it), np.std(accuracies_it)]

### DTW + 1-Knn

In [14]:
dtw_acc = []

for train_ind, test_ind in zip(train_index, test_index):
    X_train = np.vstack([clean_data[key]['X'] for key in subjects[train_ind]])
    y_train = np.hstack([clean_data[key][response] for key in subjects[train_ind]])
    
    X_test = np.vstack([clean_data[key]['X'] for key in subjects[test_ind]])
    y_test = np.hstack([clean_data[key][response] for key in subjects[test_ind]])
    
    X_train = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_train)
    X_test = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_test)
    
    dtw_knn = MultivariateClassifier(KNeighborsClassifier(metric = 'dtw_itakura',
                                                          n_jobs = -1,
                                                          metric_params = {'max_slope' : 2}))
    
    dtw_knn.fit(X_train,y_train)
    acc = dtw_knn.score(X_test,y_test)
    dtw_acc.append(acc)
    
scores['dtw_knn'] = [np.mean(dtw_acc), np.std(dtw_acc)]

### WEASELMUSE

In [15]:
#Accuracies
accuracies_lr = []
accuracies_rf = []
accuracies_svc = []
accuracies_lgbm = []

#Train and evaluate the models over the stratified splits
for train_ind, test_ind in zip(train_index, test_index):
    print(i)
    i = i+1
    X_train = np.vstack([clean_data[key]['X'] for key in subjects[train_ind]])
    y_train = np.hstack([clean_data[key][response] for key in subjects[train_ind]])
    
    X_test = np.vstack([clean_data[key]['X'] for key in subjects[test_ind]])
    y_test = np.hstack([clean_data[key][response] for key in subjects[test_ind]])

    #wm + lr
    wm = WEASELMUSE(word_size = hyperparameters['wm_lr']['word_size'], 
                    n_bins = hyperparameters['wm_lr']['n_bins'])

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['wm_lr']['C'])

    clf = make_pipeline(wm, logistic)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lr.append(np.round(acc,2))
    
    #wm + rf
    wm = WEASELMUSE(word_size = hyperparameters['wm_rf']['word_size'], 
                    n_bins = hyperparameters['wm_rf']['n_bins'])

    rf = RandomForestClassifier(n_estimators = hyperparameters['wm_rf']['n_estimators'],
                                max_depth = hyperparameters['wm_rf']['max_depth'])

    clf = make_pipeline(wm, rf)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_rf.append(np.round(acc,2))

    #wm + svc
    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    svc = SVC(C = hyperparameters['wm_svc']['C'],
              kernel = hyperparameters['wm_svc']['kernel'],
              degree = hyperparameters['wm_svc']['degree'],
              gamma = hyperparameters['wm_svc']['gamma'])

    clf = make_pipeline(wm, svc)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_svc.append(np.round(acc,2))

    #wm + lgbm
    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    def sparse_float(mat):
        return mat.astype('float')
    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['wm_lgbm']['num_leaves'],
                          max_depth = hyperparameters['wm_lgbm']['max_depth'],
                          learning_rate = hyperparameters['wm_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['wm_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['wm_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['wm_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['wm_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['wm_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['wm_lgbm']['reg_lambda'])

    clf = make_pipeline(wm, trans_sparse_float, lgbm)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lgbm.append(np.round(acc,2))
    
#Save the mean and std of accuracies of the models over the splits
scores['wm_lr'] = [np.mean(accuracies_lr), np.std(accuracies_lr)]
scores['wm_rf'] = [np.mean(accuracies_rf), np.std(accuracies_rf)]
scores['wm_svc'] = [np.mean(accuracies_svc), np.std(accuracies_svc)]
scores['wm_lgbm'] = [np.mean(accuracies_lgbm), np.std(accuracies_lgbm)]

### PyEEG

In [17]:
#Accuracies
pyeeg_lr_acc = []
pyeeg_rf_acc = []
pyeeg_svc_acc = []
pyeeg_lgbm_acc = []

#Train and evaluate the models over the stratified splits
for train_ind, test_ind in zip(train_index, test_index):

    X_train = np.vstack([clean_data[key]['X'] for key in subjects[train_ind]])
    y_train = np.hstack([clean_data[key][response] for key in subjects[train_ind]])
    
    X_test = np.vstack([clean_data[key]['X'] for key in subjects[test_ind]])
    y_test = np.hstack([clean_data[key][response] for key in subjects[test_ind]])
    
    #Create features out of the eeg channels
    feat_dict = {}

    for df,name in zip([X_train,X_test],['X_train_pyeeg','X_test_pyeeg']):
        feat_dict[name] = np.array([]).reshape(-1,2336)

        for ind in range(0,df.shape[0]):
            features = np.array([])

            for channel in range(0,df[ind].shape[0]):
                eeg_series = df[ind][channel]
                
                #Wavelet Coefficients features
                coefs = wavedec(eeg_series, 'db4', level=4)
                for band in coefs:
                    features = np.hstack([features,get_features(band)])
                
                #Fourier Transform features
                fft_power = pyeeg.bin_power(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256)
                power_spectral_intensity = fft_power[0]
                relative_intensity_ratio = fft_power[1]
                spectral_entropy = pyeeg.spectral_entropy(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256,
                                                          Power_Ratio = relative_intensity_ratio)

                #Hjorth Parameters features
                hjorth = pyeeg.hjorth(eeg_series)
                hjorth_mob = hjorth[0]
                hjorth_comp = hjorth[1]

                features = np.hstack([features, power_spectral_intensity, relative_intensity_ratio,
                                      hjorth_mob, hjorth_comp, spectral_entropy])


            feat_dict[name] = np.vstack([feat_dict[name],features])

    #Convert dictionary of features into numpy arrays
    X_train_pyeeg = feat_dict['X_train_pyeeg'].copy()
    X_test_pyeeg = feat_dict['X_test_pyeeg'].copy()
    
    X_train_pyeeg = np.nan_to_num(X_train_pyeeg)
    X_test_pyeeg = np.nan_to_num(X_test_pyeeg)
    scaler = preprocessing.StandardScaler().fit(X_train_pyeeg)
    X_train_pyeeg = scaler.transform(X_train_pyeeg)
    X_test_pyeeg = scaler.transform(X_test_pyeeg)
    
    #pyeeg + lr
    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['pyeeg_lr']['C'])

    logistic.fit(X_train_pyeeg,y_train)
    acc = logistic.score(X_test_pyeeg,y_test)
    pyeeg_lr_acc.append(np.round(acc,2).copy())
    
    #pyeeg + rf
    rf = RandomForestClassifier(n_estimators = hyperparameters['pyeeg_rf']['n_estimators'],
                                max_depth = hyperparameters['pyeeg_rf']['max_depth'])

    rf.fit(X_train_pyeeg,y_train)
    acc = rf.score(X_test_pyeeg,y_test)
    pyeeg_rf_acc.append(np.round(acc,2).copy())
    
    #pyeeg + svc
    svc = SVC(C = hyperparameters['pyeeg_svc']['C'],
              kernel = hyperparameters['pyeeg_svc']['kernel'],
              degree = hyperparameters['pyeeg_svc']['degree'],
              gamma = hyperparameters['pyeeg_svc']['gamma'])

    clf = make_pipeline(svc)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_svc_acc.append(np.round(acc,2).copy())

    
    #pyeeg + lgbm
    def sparse_float(mat):
        return mat.astype('float')
    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['pyeeg_lgbm']['num_leaves'],
                          max_depth = hyperparameters['pyeeg_lgbm']['max_depth'],
                          learning_rate = hyperparameters['pyeeg_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['pyeeg_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['pyeeg_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['pyeeg_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['pyeeg_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['pyeeg_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['pyeeg_lgbm']['reg_lambda'])

    clf = make_pipeline(trans_sparse_float, lgbm)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_lgbm_acc.append(np.round(acc,2).copy())

#Save the mean and std of accuracies of the models over the splits
scores['pyeeg_lr'] = [np.mean(pyeeg_lr_acc), np.std(pyeeg_lr_acc)]
scores['pyeeg_rf'] = [np.mean(pyeeg_rf_acc), np.std(pyeeg_rf_acc)]
scores['pyeeg_svc'] = [np.mean(pyeeg_svc_acc), np.std(pyeeg_svc_acc)]
scores['pyeeg_lgbm'] = [np.mean(pyeeg_lgbm_acc), np.std(pyeeg_lgbm_acc)]

In [ ]:
#Salva scores
if not os.path.exists(os.path.join('results')):
    os.makedirs(os.path.join('results'))

results = pd.Series({'DTW + 1NN' : scores['dtw_knn'][0],
                     'WM + LR' : scores['wm_lr'][0],
                     'WM + RF' : scores['wm_rf'][0],
                     'WM + SVM' : scores['wm_svc'][0],
                     'WM + GBT' : scores['wm_lgbm'][0],
                     'FS + LR' : scores['pyeeg_lr'][0],
                     'FS + RF' : scores['pyeeg_rf'][0],
                     'FS + SVM' : scores['pyeeg_svc'][0],
                     'FS + GBT' : scores['pyeeg_lgbm'][0],
                     'InceptionTime' : scores['dl_it'][0],
                     'CNN' : scores['dl_cnn'][0]})

results.to_csv('results/Scores_{}.csv'.format(dataset), index = False)